# 🏠 Procedural Floor Plan Generator

Generate simple 2D floor plans represented as binary arrays (0 = empty space, 1 = wall).

This notebook includes **three different algorithms** for variety:
1. **Binary Space Partitioning (BSP)** - Clean, architectural layouts
2. **Cellular Automata** - Organic, cave-like spaces
3. **Random Room Placement** - Classic dungeon/building style

Each algorithm has tunable parameters for endless variety!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import random
from typing import List, Tuple, Optional
from dataclasses import dataclass
from copy import deepcopy

# Set random seed for reproducibility (comment out for different results each time)
# np.random.seed(42)
# random.seed(42)

## Visualization Helper

A simple function to display our floor plans nicely.

In [ ]:
def display_floor_plan(grid: np.ndarray, title: str = "Floor Plan", figsize: Tuple[int, int] = (10, 10)):
    """Display a floor plan with a nice color scheme."""
    fig, ax = plt.subplots(figsize=figsize)
    
    # Custom colormap: white for empty (0), dark gray for walls (1)
    cmap = ListedColormap(['#FFFFFF', '#2C3E50'])
    
    ax.imshow(grid, cmap=cmap, interpolation='nearest')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Add grid lines for clarity
    ax.set_xticks(np.arange(-0.5, grid.shape[1], 1), minor=True)
    ax.set_yticks(np.arange(-0.5, grid.shape[0], 1), minor=True)
    ax.grid(which='minor', color='#BDC3C7', linewidth=0.5, alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
def display_multiple(grids: List[np.ndarray], titles: List[str], cols: int = 3, figsize: Tuple[int, int] = (15, 5)):
    """Display multiple floor plans side by side."""
    rows = (len(grids) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(figsize[0], figsize[1] * rows))
    axes = np.atleast_2d(axes).flatten()
    
    cmap = ListedColormap(['#FFFFFF', '#2C3E50'])
    
    for idx, (grid, title) in enumerate(zip(grids, titles)):
        axes[idx].imshow(grid, cmap=cmap, interpolation='nearest')
        axes[idx].set_title(title, fontsize=11, fontweight='bold')
        axes[idx].set_xticks([])
        axes[idx].set_yticks([])
    
    # Hide empty subplots
    for idx in range(len(grids), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

---
## Algorithm 1: Binary Space Partitioning (BSP)

BSP recursively divides space into smaller regions, creating rooms with clean rectangular shapes. Great for buildings, apartments, and structured layouts.

### How it works:
1. Start with the entire grid as one region
2. Recursively split regions horizontally or vertically
3. Stop when regions are small enough (these become rooms)
4. Connect adjacent rooms with doors

In [ ]:
@dataclass
class BSPNode:
    """A node in the BSP tree representing a rectangular region."""
    x: int
    y: int
    width: int
    height: int
    left: Optional['BSPNode'] = None
    right: Optional['BSPNode'] = None
    room: Optional[Tuple[int, int, int, int]] = None  # (x, y, w, h) of the room inside

class BSPFloorPlan:
    """Generate floor plans using Binary Space Partitioning."""
    
    def __init__(
        self,
        width: int = 50,
        height: int = 50,
        min_room_size: int = 6,
        max_room_size: int = 15,
        room_padding: int = 1,
        door_width: int = 2,
        split_variance: float = 0.3
    ):
        self.width = width
        self.height = height
        self.min_room_size = min_room_size
        self.max_room_size = max_room_size
        self.room_padding = room_padding
        self.door_width = door_width
        self.split_variance = split_variance  # How much to vary split position (0-0.5)
        
    def generate(self) -> np.ndarray:
        """Generate a new floor plan."""
        # Start with all walls
        self.grid = np.ones((self.height, self.width), dtype=np.int8)
        
        # Create BSP tree
        root = BSPNode(0, 0, self.width, self.height)
        self._split(root)
        
        # Create rooms in leaf nodes
        self._create_rooms(root)
        
        # Connect rooms
        self._connect_rooms(root)
        
        return self.grid
    
    def _split(self, node: BSPNode, depth: int = 0) -> None:
        """Recursively split a node into two children."""
        # Stop if too small
        if node.width < self.min_room_size * 2 or node.height < self.min_room_size * 2:
            return
        if max(node.width, node.height) < self.max_room_size and random.random() < 0.3:
            return  # Random early stop for variety
        
        # Decide split direction (prefer splitting the longer dimension)
        if node.width > node.height * 1.25:
            horizontal = False
        elif node.height > node.width * 1.25:
            horizontal = True
        else:
            horizontal = random.random() < 0.5
        
        # Calculate split position with variance
        if horizontal:
            max_split = node.height - self.min_room_size
            min_split = self.min_room_size
            if max_split <= min_split:
                return
            variance = int((max_split - min_split) * self.split_variance)
            mid = node.height // 2
            split = mid + random.randint(-variance, variance)
            split = max(min_split, min(max_split, split))
            
            node.left = BSPNode(node.x, node.y, node.width, split)
            node.right = BSPNode(node.x, node.y + split, node.width, node.height - split)
        else:
            max_split = node.width - self.min_room_size
            min_split = self.min_room_size
            if max_split <= min_split:
                return
            variance = int((max_split - min_split) * self.split_variance)
            mid = node.width // 2
            split = mid + random.randint(-variance, variance)
            split = max(min_split, min(max_split, split))
            
            node.left = BSPNode(node.x, node.y, split, node.height)
            node.right = BSPNode(node.x + split, node.y, node.width - split, node.height)
        
        # Recurse
        self._split(node.left, depth + 1)
        self._split(node.right, depth + 1)
    
    def _create_rooms(self, node: BSPNode) -> None:
        """Create rooms in leaf nodes."""
        if node.left is None and node.right is None:
            # Leaf node - create a room
            pad = self.room_padding
            room_x = node.x + pad
            room_y = node.y + pad
            room_w = node.width - pad * 2
            room_h = node.height - pad * 2
            
            # Add some randomness to room size
            if room_w > self.min_room_size + 2 and random.random() < 0.3:
                shrink = random.randint(1, 2)
                room_x += shrink
                room_w -= shrink * 2
            if room_h > self.min_room_size + 2 and random.random() < 0.3:
                shrink = random.randint(1, 2)
                room_y += shrink
                room_h -= shrink * 2
            
            node.room = (room_x, room_y, room_w, room_h)
            
            # Carve out the room
            for y in range(room_y, room_y + room_h):
                for x in range(room_x, room_x + room_w):
                    if 0 <= y < self.height and 0 <= x < self.width:
                        self.grid[y, x] = 0
        else:
            if node.left:
                self._create_rooms(node.left)
            if node.right:
                self._create_rooms(node.right)
    
    def _get_room(self, node: BSPNode) -> Optional[Tuple[int, int, int, int]]:
        """Get a room from this node or its descendants."""
        if node.room:
            return node.room
        
        left_room = self._get_room(node.left) if node.left else None
        right_room = self._get_room(node.right) if node.right else None
        
        if left_room and right_room:
            return random.choice([left_room, right_room])
        return left_room or right_room
    
    def _connect_rooms(self, node: BSPNode) -> None:
        """Connect rooms through the BSP tree."""
        if node.left is None or node.right is None:
            return
        
        # Recurse first
        self._connect_rooms(node.left)
        self._connect_rooms(node.right)
        
        # Connect left and right subtrees
        left_room = self._get_room(node.left)
        right_room = self._get_room(node.right)
        
        if left_room and right_room:
            self._create_corridor(left_room, right_room)
    
    def _create_corridor(self, room1: Tuple, room2: Tuple) -> None:
        """Create a corridor between two rooms."""
        # Get center points
        x1 = room1[0] + room1[2] // 2
        y1 = room1[1] + room1[3] // 2
        x2 = room2[0] + room2[2] // 2
        y2 = room2[1] + room2[3] // 2
        
        # Create L-shaped corridor
        if random.random() < 0.5:
            self._carve_h_corridor(x1, x2, y1)
            self._carve_v_corridor(y1, y2, x2)
        else:
            self._carve_v_corridor(y1, y2, x1)
            self._carve_h_corridor(x1, x2, y2)
    
    def _carve_h_corridor(self, x1: int, x2: int, y: int) -> None:
        """Carve a horizontal corridor."""
        for x in range(min(x1, x2), max(x1, x2) + 1):
            for dy in range(self.door_width):
                py = y + dy - self.door_width // 2
                if 0 <= py < self.height and 0 <= x < self.width:
                    self.grid[py, x] = 0
    
    def _carve_v_corridor(self, y1: int, y2: int, x: int) -> None:
        """Carve a vertical corridor."""
        for y in range(min(y1, y2), max(y1, y2) + 1):
            for dx in range(self.door_width):
                px = x + dx - self.door_width // 2
                if 0 <= y < self.height and 0 <= px < self.width:
                    self.grid[y, px] = 0

In [ ]:
# Demo: BSP Floor Plans with different parameters
bsp = BSPFloorPlan(width=50, height=50, min_room_size=8, split_variance=0.4)
plan = bsp.generate()
display_floor_plan(plan, "BSP Floor Plan")

In [ ]:
# Generate multiple BSP variations
bsp_plans = []
bsp_titles = []

configs = [
    {"min_room_size": 6, "split_variance": 0.2, "room_padding": 1},
    {"min_room_size": 10, "split_variance": 0.4, "room_padding": 2},
    {"min_room_size": 8, "split_variance": 0.5, "room_padding": 1},
]

for i, cfg in enumerate(configs):
    bsp = BSPFloorPlan(width=40, height=40, **cfg)
    bsp_plans.append(bsp.generate())
    bsp_titles.append(f"BSP Variant {i+1}")

display_multiple(bsp_plans, bsp_titles)

---
## Algorithm 2: Cellular Automata

Cellular automata creates organic, natural-looking spaces. Great for caves, ruins, or irregular buildings.

### How it works:
1. Start with random noise (each cell randomly wall or empty)
2. Apply smoothing rules iteratively:
   - If a cell has many wall neighbors, it becomes a wall
   - If a cell has few wall neighbors, it becomes empty
3. The result is smooth, organic shapes

In [ ]:
class CellularAutomataFloorPlan:
    """Generate floor plans using Cellular Automata - optimized for airy layouts."""

    def __init__(
        self,
        width: int = 50,
        height: int = 50,
        fill_probability: float = 0.35,  # Lower default for more open space
        iterations: int = 4,
        wall_threshold: int = 5,
        birth_threshold: int = 6,  # Higher = harder to create new walls
        add_border: bool = True,
        remove_small_walls: bool = True,  # Remove isolated wall clusters
        min_wall_cluster_size: int = 15,  # Minimum wall cluster to keep
        thin_walls: bool = True,  # Erode thick walls
    ):
        self.width = width
        self.height = height
        self.fill_probability = fill_probability
        self.iterations = iterations
        self.wall_threshold = wall_threshold
        self.birth_threshold = birth_threshold
        self.add_border = add_border
        self.remove_small_walls = remove_small_walls
        self.min_wall_cluster_size = min_wall_cluster_size
        self.thin_walls = thin_walls

    def generate(self) -> np.ndarray:
        """Generate a new floor plan."""
        # Initialize with random noise - biased toward empty
        self.grid = np.random.choice(
            [0, 1],
            size=(self.height, self.width),
            p=[1 - self.fill_probability, self.fill_probability]
        ).astype(np.int8)

        # Apply cellular automata rules
        for _ in range(self.iterations):
            self._iterate()

        # Post-processing for airier layouts
        if self.remove_small_walls:
            self._remove_small_wall_clusters()

        if self.thin_walls:
            self._erode_thick_walls()

        # Add border walls
        if self.add_border:
            self.grid[0, :] = 1
            self.grid[-1, :] = 1
            self.grid[:, 0] = 1
            self.grid[:, -1] = 1

        return self.grid

    def _count_neighbors(self, x: int, y: int) -> int:
        """Count wall neighbors (including diagonals)."""
        count = 0
        for dy in range(-1, 2):
            for dx in range(-1, 2):
                if dx == 0 and dy == 0:
                    continue
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.width and 0 <= ny < self.height:
                    count += self.grid[ny, nx]
                else:
                    count += 1  # Out of bounds counts as wall
        return count

    def _iterate(self) -> None:
        """Apply one iteration of cellular automata rules."""
        new_grid = np.zeros_like(self.grid)

        for y in range(self.height):
            for x in range(self.width):
                neighbors = self._count_neighbors(x, y)

                if self.grid[y, x] == 1:
                    # Wall survives if enough neighbors
                    new_grid[y, x] = 1 if neighbors >= self.wall_threshold else 0
                else:
                    # Empty becomes wall if enough neighbors (higher threshold = fewer new walls)
                    new_grid[y, x] = 1 if neighbors >= self.birth_threshold else 0

        self.grid = new_grid

    def _flood_fill_walls(self, start_x: int, start_y: int, visited: np.ndarray) -> List[Tuple[int, int]]:
        """Flood fill to find connected wall cluster."""
        cluster = []
        stack = [(start_x, start_y)]

        while stack:
            x, y = stack.pop()
            if visited[y, x] or self.grid[y, x] == 0:
                continue

            visited[y, x] = True
            cluster.append((x, y))

            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.width and 0 <= ny < self.height:
                    if not visited[ny, nx] and self.grid[ny, nx] == 1:
                        stack.append((nx, ny))

        return cluster

    def _remove_small_wall_clusters(self) -> None:
        """Remove isolated wall clusters smaller than threshold."""
        visited = np.zeros((self.height, self.width), dtype=bool)

        for y in range(self.height):
            for x in range(self.width):
                if self.grid[y, x] == 1 and not visited[y, x]:
                    cluster = self._flood_fill_walls(x, y, visited)
                    if len(cluster) < self.min_wall_cluster_size:
                        # Remove this small cluster
                        for cx, cy in cluster:
                            self.grid[cy, cx] = 0

    def _erode_thick_walls(self) -> None:
        """Erode thick wall masses to create thinner walls like BSP."""
        # Multiple erosion passes to thin out walls
        for _ in range(2):
            new_grid = self.grid.copy()
            for y in range(1, self.height - 1):
                for x in range(1, self.width - 1):
                    if self.grid[y, x] == 1:
                        # Count empty neighbors (4-connected)
                        empty_neighbors = sum(
                            self.grid[y + dy, x + dx] == 0
                            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
                        )
                        # If completely surrounded by walls, remove it
                        if empty_neighbors == 0:
                            new_grid[y, x] = 0
            self.grid = new_grid

In [ ]:
# Demo: Cellular Automata Floor Plan
ca = CellularAutomataFloorPlan(width=60, height=60, fill_probability=0.45, iterations=5)
plan = ca.generate()
display_floor_plan(plan, "Cellular Automata Floor Plan")

In [ ]:
# Show the evolution of cellular automata
ca = CellularAutomataFloorPlan(width=40, height=40, fill_probability=0.45, iterations=1)
evolution = []
titles = []

# Get initial state
np.random.seed(123)  # For consistent demo
ca.grid = np.random.choice([0, 1], size=(40, 40), p=[0.55, 0.45]).astype(np.int8)
evolution.append(ca.grid.copy())
titles.append("Initial (Random)")

# Show iterations
for i in range(4):
    ca._iterate()
    evolution.append(ca.grid.copy())
    titles.append(f"Iteration {i+1}")

display_multiple(evolution, titles, cols=5, figsize=(18, 4))

In [ ]:
# Different CA variations - now airier!
ca_plans = []
ca_titles = []

configs = [
    {"fill_probability": 0.30, "iterations": 4, "wall_threshold": 5, "birth_threshold": 6},
    {"fill_probability": 0.35, "iterations": 5, "wall_threshold": 5, "birth_threshold": 7},
    {"fill_probability": 0.32, "iterations": 4, "wall_threshold": 4, "birth_threshold": 6},
]

for i, cfg in enumerate(configs):
    ca = CellularAutomataFloorPlan(width=45, height=45, **cfg)
    ca_plans.append(ca.generate())
    ca_titles.append(f"CA Variant {i+1}")

display_multiple(ca_plans, ca_titles)

---
## Algorithm 3: Random Room Placement

Places random rectangular rooms and connects them with corridors. Classic dungeon/RPG style.

### How it works:
1. Attempt to place random rooms that don't overlap
2. Connect each room to the nearest unconnected room
3. Optionally add extra connections for loops

In [ ]:
class RandomRoomFloorPlan:
    """Generate floor plans by placing random rooms and connecting them - optimized for airy layouts."""

    def __init__(
        self,
        width: int = 50,
        height: int = 50,
        num_rooms: int = 12,  # More rooms by default
        min_room_size: int = 6,
        max_room_size: int = 14,
        room_margin: int = 1,  # Reduced margin for tighter packing
        corridor_width: int = 2,
        extra_connections: int = 2,
        max_attempts: int = 100,
        fill_remaining: bool = True,  # Try to fill empty spaces with more rooms
        min_fill_room_size: int = 4,  # Minimum size for fill rooms
    ):
        self.width = width
        self.height = height
        self.num_rooms = num_rooms
        self.min_room_size = min_room_size
        self.max_room_size = max_room_size
        self.room_margin = room_margin
        self.corridor_width = corridor_width
        self.extra_connections = extra_connections
        self.max_attempts = max_attempts
        self.fill_remaining = fill_remaining
        self.min_fill_room_size = min_fill_room_size

    def generate(self) -> np.ndarray:
        """Generate a new floor plan."""
        # Start with all walls
        self.grid = np.ones((self.height, self.width), dtype=np.int8)
        self.rooms = []

        # Place main rooms
        self._place_rooms()

        # Fill remaining space with smaller rooms
        if self.fill_remaining:
            self._fill_remaining_space()

        # Connect rooms
        self._connect_rooms()

        # Add extra connections for loops
        self._add_extra_connections()

        return self.grid

    def _place_rooms(self) -> None:
        """Try to place the requested number of rooms."""
        for _ in range(self.num_rooms):
            for _ in range(self.max_attempts):
                # Random room size
                w = random.randint(self.min_room_size, self.max_room_size)
                h = random.randint(self.min_room_size, self.max_room_size)

                # Random position
                x = random.randint(1, self.width - w - 1)
                y = random.randint(1, self.height - h - 1)

                # Check for overlap (with margin)
                if not self._check_overlap(x, y, w, h):
                    self._carve_room(x, y, w, h)
                    self.rooms.append((x, y, w, h))
                    break

    def _fill_remaining_space(self) -> None:
        """Fill empty wall regions with additional smaller rooms."""
        # Try to place many small rooms to fill gaps
        fill_attempts = self.num_rooms * 3
        for _ in range(fill_attempts):
            for _ in range(self.max_attempts // 2):
                # Smaller rooms for filling
                w = random.randint(self.min_fill_room_size, self.min_room_size + 2)
                h = random.randint(self.min_fill_room_size, self.min_room_size + 2)

                x = random.randint(1, self.width - w - 1)
                y = random.randint(1, self.height - h - 1)

                if not self._check_overlap(x, y, w, h, margin=1):
                    self._carve_room(x, y, w, h)
                    self.rooms.append((x, y, w, h))
                    break

    def _check_overlap(self, x: int, y: int, w: int, h: int, margin: int = None) -> bool:
        """Check if a room would overlap with existing rooms."""
        m = margin if margin is not None else self.room_margin
        for rx, ry, rw, rh in self.rooms:
            if (x - m < rx + rw and x + w + m > rx and
                y - m < ry + rh and y + h + m > ry):
                return True
        return False

    def _carve_room(self, x: int, y: int, w: int, h: int) -> None:
        """Carve out a room in the grid."""
        for py in range(y, y + h):
            for px in range(x, x + w):
                if 0 <= py < self.height and 0 <= px < self.width:
                    self.grid[py, px] = 0

    def _room_center(self, room: Tuple) -> Tuple[int, int]:
        """Get the center of a room."""
        return (room[0] + room[2] // 2, room[1] + room[3] // 2)

    def _connect_rooms(self) -> None:
        """Connect all rooms using nearest neighbor approach."""
        if len(self.rooms) < 2:
            return

        connected = {0}
        unconnected = set(range(1, len(self.rooms)))

        while unconnected:
            best_dist = float('inf')
            best_pair = None

            for ci in connected:
                for ui in unconnected:
                    c1 = self._room_center(self.rooms[ci])
                    c2 = self._room_center(self.rooms[ui])
                    dist = abs(c1[0] - c2[0]) + abs(c1[1] - c2[1])
                    if dist < best_dist:
                        best_dist = dist
                        best_pair = (ci, ui)

            if best_pair:
                self._create_corridor(self.rooms[best_pair[0]], self.rooms[best_pair[1]])
                connected.add(best_pair[1])
                unconnected.remove(best_pair[1])

    def _add_extra_connections(self) -> None:
        """Add extra corridors for loops."""
        if len(self.rooms) < 3:
            return

        for _ in range(self.extra_connections):
            r1, r2 = random.sample(self.rooms, 2)
            self._create_corridor(r1, r2)

    def _create_corridor(self, room1: Tuple, room2: Tuple) -> None:
        """Create a corridor between two rooms."""
        c1 = self._room_center(room1)
        c2 = self._room_center(room2)

        # L-shaped corridor
        if random.random() < 0.5:
            self._carve_h_corridor(c1[0], c2[0], c1[1])
            self._carve_v_corridor(c1[1], c2[1], c2[0])
        else:
            self._carve_v_corridor(c1[1], c2[1], c1[0])
            self._carve_h_corridor(c1[0], c2[0], c2[1])

    def _carve_h_corridor(self, x1: int, x2: int, y: int) -> None:
        """Carve a horizontal corridor."""
        for x in range(min(x1, x2), max(x1, x2) + 1):
            for dy in range(self.corridor_width):
                py = y + dy - self.corridor_width // 2
                if 0 <= py < self.height and 0 <= x < self.width:
                    self.grid[py, x] = 0

    def _carve_v_corridor(self, y1: int, y2: int, x: int) -> None:
        """Carve a vertical corridor."""
        for y in range(min(y1, y2), max(y1, y2) + 1):
            for dx in range(self.corridor_width):
                px = x + dx - self.corridor_width // 2
                if 0 <= y < self.height and 0 <= px < self.width:
                    self.grid[y, px] = 0

In [ ]:
# Demo: Random Room Floor Plan
rr = RandomRoomFloorPlan(width=60, height=60, num_rooms=10, extra_connections=3)
plan = rr.generate()
display_floor_plan(plan, "Random Room Floor Plan")

In [ ]:
# Different Random Room variations - now airier with fill_remaining!
rr_plans = []
rr_titles = []

configs = [
    {"num_rooms": 8, "min_room_size": 8, "max_room_size": 15, "fill_remaining": True},
    {"num_rooms": 15, "min_room_size": 5, "max_room_size": 10, "fill_remaining": True},
    {"num_rooms": 10, "min_room_size": 6, "max_room_size": 12, "corridor_width": 3, "fill_remaining": True},
]

for i, cfg in enumerate(configs):
    rr = RandomRoomFloorPlan(width=45, height=45, **cfg)
    rr_plans.append(rr.generate())
    rr_titles.append(f"Rooms Variant {i+1}")

display_multiple(rr_plans, rr_titles)

---
## Unified Generator

A convenience class that can generate floor plans using any algorithm with random parameters for maximum variety.

In [ ]:
class FloorPlanGenerator:
    """Unified generator that can use any algorithm - optimized for airy layouts."""

    ALGORITHMS = ['bsp', 'cellular', 'rooms']

    def __init__(self, width: int = 50, height: int = 50):
        self.width = width
        self.height = height

    def generate(self, algorithm: str = 'random', **kwargs) -> np.ndarray:
        """
        Generate a floor plan.

        Args:
            algorithm: 'bsp', 'cellular', 'rooms', or 'random'
            **kwargs: Additional parameters for the chosen algorithm

        Returns:
            2D numpy array where 0=empty, 1=wall
        """
        if algorithm == 'random':
            algorithm = random.choice(self.ALGORITHMS)

        if algorithm == 'bsp':
            gen = BSPFloorPlan(self.width, self.height, **kwargs)
        elif algorithm == 'cellular':
            gen = CellularAutomataFloorPlan(self.width, self.height, **kwargs)
        elif algorithm == 'rooms':
            gen = RandomRoomFloorPlan(self.width, self.height, **kwargs)
        else:
            raise ValueError(f"Unknown algorithm: {algorithm}")

        return gen.generate()

    def generate_varied(self, algorithm: str = 'random') -> np.ndarray:
        """
        Generate with randomized parameters for extra variety.
        All algorithms now optimized for airy, open layouts.
        """
        if algorithm == 'random':
            algorithm = random.choice(self.ALGORITHMS)

        if algorithm == 'bsp':
            kwargs = {
                'min_room_size': random.randint(5, 10),
                'room_padding': random.randint(1, 2),
                'split_variance': random.uniform(0.2, 0.5),
                'door_width': random.randint(1, 3)
            }
        elif algorithm == 'cellular':
            # Airy CA parameters
            kwargs = {
                'fill_probability': random.uniform(0.28, 0.38),
                'iterations': random.randint(3, 5),
                'wall_threshold': random.randint(4, 5),
                'birth_threshold': random.randint(6, 7),
                'remove_small_walls': True,
                'thin_walls': True,
            }
        elif algorithm == 'rooms':
            # Airy rooms parameters
            kwargs = {
                'num_rooms': random.randint(10, 15),
                'min_room_size': random.randint(5, 7),
                'max_room_size': random.randint(12, 16),
                'corridor_width': random.randint(2, 3),
                'extra_connections': random.randint(2, 5),
                'fill_remaining': True,
                'room_margin': 1,
            }

        return self.generate(algorithm, **kwargs)

In [ ]:
# Generate a gallery of varied floor plans
gen = FloorPlanGenerator(width=45, height=45)

plans = []
titles = []

for algo in ['bsp', 'bsp', 'cellular', 'cellular', 'rooms', 'rooms']:
    plans.append(gen.generate_varied(algo))
    titles.append(algo.upper())

display_multiple(plans, titles, cols=3, figsize=(15, 10))

---
## Quick Export Utility

Simple functions to export your floor plans.

In [ ]:
def save_floor_plan(grid: np.ndarray, filename: str):
    """Save floor plan to various formats."""
    if filename.endswith('.npy'):
        np.save(filename, grid)
    elif filename.endswith('.txt'):
        np.savetxt(filename, grid, fmt='%d')
    elif filename.endswith('.png'):
        fig, ax = plt.subplots(figsize=(10, 10))
        cmap = ListedColormap(['#FFFFFF', '#2C3E50'])
        ax.imshow(grid, cmap=cmap, interpolation='nearest')
        ax.axis('off')
        plt.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=150)
        plt.close()
    else:
        raise ValueError("Supported formats: .npy, .txt, .png")
    print(f"Saved to {filename}")

def load_floor_plan(filename: str) -> np.ndarray:
    """Load a floor plan from file."""
    if filename.endswith('.npy'):
        return np.load(filename)
    elif filename.endswith('.txt'):
        return np.loadtxt(filename, dtype=np.int8)
    else:
        raise ValueError("Supported formats: .npy, .txt")

In [ ]:
# Example: Generate and save
gen = FloorPlanGenerator(50, 50)
plan = gen.generate('bsp')

# Uncomment to save:
# save_floor_plan(plan, 'my_floor_plan.npy')
# save_floor_plan(plan, 'my_floor_plan.txt')
# save_floor_plan(plan, 'my_floor_plan.png')

---
## 🎮 Interactive Playground

Play around with different parameters!

In [ ]:
# === CUSTOMIZE YOUR FLOOR PLAN HERE ===

WIDTH = 50
HEIGHT = 50
ALGORITHM = 'bsp'  # Options: 'bsp', 'cellular', 'rooms'

# BSP parameters
BSP_MIN_ROOM = 7
BSP_PADDING = 1
BSP_VARIANCE = 0.4

# Cellular Automata parameters  
CA_FILL = 0.45
CA_ITERATIONS = 5

# Random Rooms parameters
RR_NUM_ROOMS = 8
RR_MIN_SIZE = 5
RR_MAX_SIZE = 12

# Generate!
gen = FloorPlanGenerator(WIDTH, HEIGHT)

if ALGORITHM == 'bsp':
    plan = gen.generate('bsp', min_room_size=BSP_MIN_ROOM, room_padding=BSP_PADDING, split_variance=BSP_VARIANCE)
elif ALGORITHM == 'cellular':
    plan = gen.generate('cellular', fill_probability=CA_FILL, iterations=CA_ITERATIONS)
else:
    plan = gen.generate('rooms', num_rooms=RR_NUM_ROOMS, min_room_size=RR_MIN_SIZE, max_room_size=RR_MAX_SIZE)

display_floor_plan(plan, f"{ALGORITHM.upper()} Floor Plan ({WIDTH}x{HEIGHT})")

# Print the raw array if you want to see it
# print(plan)

---
## Summary

| Algorithm | Style | Best For |
|-----------|-------|----------|
| **BSP** | Clean, rectangular | Buildings, apartments, offices |
| **Cellular Automata** | Organic, irregular | Caves, ruins, natural spaces |
| **Random Rooms** | Classic dungeon | Games, RPG maps |

**Tips for variety:**
- Adjust room sizes and counts
- Change corridor widths
- Modify iteration counts (for CA)
- Add post-processing (flood fill to remove isolated areas, add furniture, etc.)

Happy generating! 🏗️